In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import datetime
from sklearn.linear_model import LinearRegression

In [ ]:
baseflow = pd.read_csv('RRCA_baseflow.csv')

In [ ]:
result = []
for date in baseflow['Date']:
        excel_date = int(date)
        py_date = datetime.datetime(1, 1, 1) + datetime.timedelta(excel_date - 2)

        result.append(py_date)

baseflow['formatDate'] = result

display(baseflow)

In [ ]:
plt.figure(figsize=(20,15))
sns.regplot(data=baseflow.sample(frac=.25), x='Evapotranspiration', y='Observed')
plt.figure(figsize=(20,15))
sns.regplot(data=baseflow.sample(frac=.25), x='Precipitation', y='Observed')
plt.figure(figsize=(20,15))
sns.regplot(data=baseflow.sample(frac=.25), x='Irrigation_pumping', y='Observed')

In [ ]:
plt.figure(figsize=(30, 15))
ax = sns.regplot(x='Date', y='Evapotranspiration', data=baseflow.sample(frac=.25))


In [ ]:
plt.figure(figsize=(30, 15))
ax = sns.regplot(x='Date', y='Precipitation', data=baseflow.sample(frac=.25))

In [ ]:
plt.figure(figsize=(30, 15))
ax = sns.regplot(x='Date', y='Irrigation_pumping', data=baseflow.sample(frac=.25))

In [ ]:
plt.figure(figsize=(30, 15))
ax = sns.regplot(x='Date', y='Observed', data=baseflow.sample(frac=.25))

In [ ]:
X = baseflow[['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']]
y = baseflow.Observed

lm = LinearRegression()
lm.fit(X, y)

display(lm.intercept_)
display(list(zip(['Evapotranspiration', 'Precipitation', 'Irrigation_pumping'], lm.coef_)))

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ Evapotranspiration', data=baseflow).fit()
lm.pvalues

In [ ]:
lm = smf.ols(formula='Observed ~ Precipitation', data=baseflow).fit()
lm.pvalues

In [ ]:
lm = smf.ols(formula='Observed ~ Irrigation_pumping', data=baseflow).fit()
lm.pvalues

In [ ]:
X = baseflow[['Evapotranspiration', 'Precipitation']]
y = baseflow.Observed

lm = LinearRegression()
lm.fit(X, y)

display(lm.intercept_)
display(list(zip(['Evapotranspiration', 'Precipitation'], lm.coef_)))

In [ ]:
locationAVG = baseflow.groupby(['Segment_id'])[['Evapotranspiration', 'Precipitation', 'Irrigation_pumping', 'Observed']].mean()
locationMED = baseflow.groupby(['Segment_id'])[['Evapotranspiration', 'Precipitation', 'Irrigation_pumping', 'Observed']].median()

display(locationAVG)
display(locationMED)

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.regplot(x='Evapotranspiration', y='Observed', data=locationAVG)
plt.figure(figsize=(15, 8))
ax = sns.regplot(x='Precipitation', y='Observed', data=locationAVG)
plt.figure(figsize=(15, 8))
ax = sns.regplot(x='Irrigation_pumping', y='Observed', data=locationAVG)

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.regplot(x='Evapotranspiration', y='Observed', data=locationMED)
plt.figure(figsize=(15, 8))
ax = sns.regplot(x='Precipitation', y='Observed', data=locationMED)
plt.figure(figsize=(15, 8))
ax = sns.regplot(x='Irrigation_pumping', y='Observed', data=locationMED)